In [121]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler

In [122]:
# Load your dataset
df = pd.read_csv("final_dataset.csv", parse_dates=["Date"], index_col="Date")

In [123]:
# Select relevant features
features = ["MACD_Signal", "Label", "RSI", "EMA_50"]
target = 'Zamkniecie'

X = df[features]
y = df[[target]]

In [124]:
# Normalize the data
# === 3. Skalowanie danych ===
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

In [125]:
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

In [126]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_scaled, test_size=0.2, shuffle=False
)

In [127]:
# === 5. Budowa modelu ===
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))  # regresja — jedna wartość

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=32)

Epoch 1/50


C:\Users\lzloc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0115
Epoch 2/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0089
Epoch 3/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0086
Epoch 4/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0088
Epoch 5/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0091
Epoch 6/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0089
Epoch 7/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0087
Epoch 8/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0089
Epoch 9/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0087
Epoch 10/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0089
Epoch 11/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0091
Epoch 12/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0089
Epoch 13/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0087
Epoch 14/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0088
Epoch 15/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0091


In [128]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=3, batch_size=32)

Epoch 1/3
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0088
Epoch 2/3
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0089
Epoch 3/3
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0089


In [136]:
# === 6. Predykcje ===
predictions_scaled = model.predict(X_test)
predictions = scaler_y.inverse_transform(predictions_scaled).flatten()

55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [145]:
# === 7. Przygotowanie DataFrame z wynikami ===
df_test = df.iloc[len(X_train):].copy()
df_test = df_test.iloc[:len(predictions)]
df_test['Predicted'] = predictions

# === 8. Generowanie sygnałów ===
threshold = 0.5 # Example: 2% threshold
df_test['Signal'] = np.where(
    df_test['Predicted'] > df_test['Zamkniecie'] * (1 + threshold), 'Buy',
    np.where(df_test['Predicted'] < df_test['Zamkniecie'] * (1 - threshold), 'Sell', 'Keep')
)
df_test.loc[
    (df_test['Predicted'] - df_test['Zamkniecie']).abs() < threshold,
    'Signal'
] = 'Keep'

# === 9. Wynik ===
print(df_test[['Zamkniecie', 'Predicted', 'Signal']])

            Zamkniecie    Predicted Signal
Date                                      
2017-12-29     2673.61  1115.773926   Sell
2018-01-02     2695.81  1115.773926   Sell
2018-01-03     2713.06  1115.773926   Sell
2018-01-04     2723.99  1115.773926   Sell
2018-01-05     2743.15  1115.773926   Sell
...                ...          ...    ...
2024-12-24     6040.04  1115.773926   Sell
2024-12-26     6037.59  1115.773926   Sell
2024-12-27     5970.84  1115.773926   Sell
2024-12-30     5906.94  1115.773926   Sell
2024-12-31     5881.63  1115.773926   Sell

[1749 rows x 3 columns]


In [146]:
# Check how many signalss sell/buy
print(df_test['Signal'].value_counts())

Signal
Sell    1749
Name: count, dtype: int64


In [148]:
# === 9. Przywrócenie oryginalnej skali dla y_pred i y_test ===
y_test_inv = scaler_y.inverse_transform(y_test)
y_pred_inv = scaler_y.inverse_transform(y_pred)

NameError: name 'y_pred' is not defined

In [149]:
# === 10. Analiza wyników ===
# Uzyskaj classification report
print("Classification Report:")
print(classification_report(y_test_inv, y_pred_inv))

# Oblicz dokładność
accuracy = accuracy_score(y_test_inv, y_pred_inv)
print(f"Accuracy: {accuracy:.2f}")

Classification Report:


NameError: name 'classification_report' is not defined

In [132]:
# Saving dataset with recommendation - v2
df_test.to_csv('dataset_recommendation_v2.csv', index=False)